#### 导入模块

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input
from keras.optimizers import SGD
from keras.models import Model
import numpy as np

#### 准备数据

In [2]:
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), 
                                     num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), 
                                    num_classes=10)

#### 定义模型
##### 定义模型的架构

In [26]:
# Sequential
model = Sequential()
model.add(Input(shape=(20,)))
#model.add(Input(shape=(20,),batch_size=100)) 
# 尽量不要在定义模型的时候设置batch_size，在训练的时候设置batch_size就可以了
#在定义模型的时候设置batch_size，在训练模型的时候还需要设置batch_size。并且需要保证
# 训练的batch_size恒等于设定的batch_size,对于样本集不是整除batch_size的情况会报错
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# model
inputs = Input(shape=(784,))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

##### 定义模型优化方法和损失函数

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# loss function, optimize, metrics
model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])

#### 训练模型

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=100)

#### 评估模型

In [ ]:
score = model.evaluate(x_valid, y_valid, batch_size=128)

#### 使用模型

In [ ]:
predict = model.predict(x_test, batch_size=128)

#### 模型保存

In [ ]:
from keras.models import load_model
## creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')  

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [ ]:
# 模型克隆
from keras.models import clone_model
'''clone_model 完全复制了原模型模型的结构，并重新构建了一个模型，
但没有复制原模型的权重的值。也就是说，
对于同样的输入，model1.predict 和 model2.predict 的结果是不一样的。'''
model2 = clone_model(model1)
model2.set_weights(K.batch_get_value(model1.weights))

#### 使用内置模型

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
# Usage examples for image classification models
model = ResNet50(weights='imagenet')
x = preprocess_input(x)
preds = model.predict(x)
print(decode_predictions(preds, top = 3)[0])

# Extract features from an arbitrary intermediate layer with VGG19
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
base_model = VGG19(weights = 'imagenet')
model = Model(inputs = base_model.input, 
              outputs = base_model.get_layer('block4_pool'))

# Fine-tune InceptionV3 on a new set of classes
from keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
prediction = Dense(200, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = prediction)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy')
model.fit_generator(...)